In [33]:
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from requests_html import HTMLSession
from selenium import webdriver
#from  pyvirtualdisplay import Display
import os
from queue import Queue
import threading
from selenium.webdriver.common.keys import Keys

In [34]:
def get_content():
    positioncontent = driver.find_element_by_class_name("position-content")

    job_name = positioncontent.find_element_by_class_name('name').text

    job_request = positioncontent.find_element_by_class_name('job_request').find_element_by_tag_name('p').text
    #job_request
    position_label= positioncontent.find_element_by_class_name('position-label').text
    #position_label
    job_detail = driver.find_element_by_id("job_detail")
    job_advantage = job_detail.find_element_by_class_name('job-advantage').find_element_by_tag_name('p').text
    #job_advantage
    description = job_detail.find_element_by_class_name('job_bt').text
    #description
    work_addr = job_detail.find_element_by_class_name('work_addr').text
    work_addr = work_addr[:-5].strip()
    #work_addr
    job_company = driver.find_element_by_id('job_company')
    dict_comp = {}
    comp_name = job_company.find_element_by_class_name('fl').text
    lg_renzheng = job_company.find_element_by_class_name('dn').text
    comp_name = comp_name.replace(lg_renzheng,'').strip()
    for li in job_company.find_elements_by_tag_name('li'):
        key = li.find_element_by_class_name('hovertips').get_attribute("innerHTML")
        value = li.text.replace(key,'').strip()
        dict_comp[key]=value

    comp_domain = dict_comp.get('领域','')
    comp_stage = dict_comp.get('发展阶段','')
    comp_scale = dict_comp.get('规模','')
    comp_invest = dict_comp.get('投资机构','')
    lagou_page = job_company.find_element_by_partial_link_text(comp_name).get_attribute('href')
    comp_index_page = job_company.find_element_by_css_selector('.icon-glyph-home+a').get_attribute('href')
    job_data = job_name,job_request,position_label,job_advantage,description,work_addr,\
    comp_name,comp_domain,comp_stage,comp_scale,comp_invest,lagou_page,comp_index_page,driver.current_url
    job_data = [i.replace(',','，').replace('\n','<p>') for i in job_data]
    job_data = ','.join(job_data)+'\n'
    print(job_name,",",comp_name,',',driver.current_url)
    return job_data

In [35]:
def read_page():
    while True:
        if time.time() - get_time[handle]>5:
            try:
                curr_element_id = driver.find_element_by_id("job_detail").id
                if last_ids[driver.current_window_handle] != curr_element_id:
                    job_data = get_content()
                    data_file.write(job_data)
                    data_file.flush()
                    last_ids[handle] = curr_element_id
                    not_read.remove(driver.current_url)
                    break
            except Exception as e:
                print(driver.current_url,e)
                not_read.append(driver.current_url)
                Q.append(driver.current_url)
        else:
            time.sleep(1)

In [36]:
def get_new_url(url,handle):
    try:
        driver.get(url)
        print('get',url)
        get_time[handle] = time.time()
    except Exception as e:
        print(url,e)
        Q.append(url)

In [58]:
chromedriver = "/home/xu/Desktop/flask2/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
#display = Display(visible=0, size=(1600,600))
#display.start()
if locals().get('driver'):
    if hasattr(driver,'quit'):
        driver.quit()
driver = webdriver.Chrome(chromedriver,port = 9515)

driver.maximize_window()
#URL = "https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98?px=default&city=%E5%85%A8%E5%9B%BD#filterBox"

In [59]:
for i in range(9):
    driver.execute_script("window.open('https://www.lagou.com');")

In [60]:
with open('search_data.log') as f_all:
    Q_all = set([i.strip() for i in f_all])
    
with open('datamining_beijing.log') as f_bj:
    Q_bj = set([i.strip() for i in f_bj])

In [61]:
Q = list(Q_bj - Q_all)
not_read = Q.copy()

In [ ]:
with open('datamining_beijing.log','r') as f_read:
    Q = [line.strip() for line in f_read]
    not_read = Q.copy()

In [62]:
data_file = open('job_datamining_beijing.csv','w')
data_file.write('job_name,job_request,position_label,job_advantage,description,work_addr,comp_name,comp_domain,comp_stage,comp_scale,comp_invest,lagou_comp_page,comp_index_page,lg_url\n')

last_ids = {}
get_time = {}

In [63]:
for handle in driver.window_handles:
    last_ids[handle] = ''
    url = Q.pop()
    driver.switch_to.window(handle)
    get_new_url(url,handle)

get https://www.lagou.com/jobs/4046375.html
get https://www.lagou.com/jobs/4056953.html
get https://www.lagou.com/jobs/2853421.html
get https://www.lagou.com/jobs/4074586.html
get https://www.lagou.com/jobs/4171197.html
get https://www.lagou.com/jobs/2113031.html
get https://www.lagou.com/jobs/4019667.html
get https://www.lagou.com/jobs/3747809.html
get https://www.lagou.com/jobs/3946235.html
get https://www.lagou.com/jobs/3803907.html


In [64]:
while True:
    for handle in driver.window_handles:
        driver.switch_to.window(handle)
        #--------取当前页面数据
        if driver.current_url in not_read:
            read_page()
#-----------加载新页面---------------
#                --判断Q是否为空
        if Q:
            url = Q.pop()
            get_new_url(url,handle)
    if (not Q) and (not not_read):
        print('done!!!')
        data_file.close()
        break

数据挖掘（实习／校招） , 数美 , https://www.lagou.com/jobs/4046375.html
get https://www.lagou.com/jobs/3552647.html
数据挖掘工程师 , 鼎阅 , https://www.lagou.com/jobs/4056953.html
get https://www.lagou.com/jobs/3507966.html
数据挖掘工程师（自然语言... , 凤凰金融 , https://www.lagou.com/jobs/2853421.html
get https://www.lagou.com/jobs/3615239.html
图像数据挖掘工程师（美颜... , 美图公司 , https://www.lagou.com/jobs/4074586.html
get https://www.lagou.com/jobs/3978119.html
数据/数据挖掘工程师 , 小米 , https://www.lagou.com/jobs/4171197.html
get https://www.lagou.com/jobs/4071058.html
推荐算法/机器学习/数据挖... , 网易 , https://www.lagou.com/jobs/2113031.html
get https://www.lagou.com/jobs/3866021.html
搜狗输入法-资深数据挖掘... , 搜狗 , https://www.lagou.com/jobs/4019667.html
get https://www.lagou.com/jobs/3906658.html
餐饮生态数据挖掘资深工程师 , 美团点评 , https://www.lagou.com/jobs/3747809.html
get https://www.lagou.com/jobs/3566886.html
风控反欺诈数据挖掘&... , 百度 , https://www.lagou.com/jobs/3946235.html
get https://www.lagou.com/jobs/4143589.html
地图内容生态部_数据挖掘... , 百度 , https://www.lagou.com/jobs/3

In [42]:
Q

[]

In [43]:
not_read

['https://www.lagou.com/jobs/4024026.html']

In [ ]:
import pandas as pd

In [67]:
dfnew = pd.read_csv('job_datamining_beijing.csv')

In [68]:
dfnew

,job_name,job_request,position_label,job_advantage,description,work_addr,comp_name,comp_domain,comp_stage,comp_scale,comp_invest,lagou_comp_page,comp_index_page,lg_url
0,数据挖掘（实习／校招）,3k-6k /北京 / 经验不限 / 本科及以上 / 全职,广告营销<p>算法<p>数据挖掘<p>机器学习<p>深度学习<p>大数据,大牛导师，技术氛围好，海量数据，飞速发展,职位描述：<p>职位职责： <p>1、通过海量数据挖掘、机器学习等方法，构建用户画像、个性化...,北京 - 朝阳区 - 望京 - 望京诚盈中心,数美,企业服务，数据服务,B轮,150-500人,数美科技获1000万美元A轮融资，顺为资本、清流资本、百度风投、360投资(A轮),https://www.lagou.com/gongsi/82991.html,http://www.ishumei.com/,https://www.lagou.com/jobs/4046375.html
1,数据挖掘工程师,15k-30k /北京 / 经验3-5年 / 本科及以上 / 全职,hadoop<p>数据挖掘<p>大数据<p>数据仓库,16薪，出国游，各种补助，年底奖金,职位描述：<p>任职要求<p>1、 有3年以上大数据体系ETL系统经验，搭建过自有数据清洗...,北京 - 海淀区 - 西小口路66号东升科技园B6号楼A座8层,鼎阅,移动互联网,不需要融资,50-150人,NaN,https://www.lagou.com/gongsi/170348.html,http://www.qingoo.cn/,https://www.lagou.com/jobs/4056953.html
2,数据挖掘工程师（自然语言...,20k-40k /北京 / 经验3-5年 / 学历不限 / 全职,高级<p>NLP<p>算法<p>语音处理<p>数据挖掘<p>自然语言处理,绩效奖金，七险一金，带薪假期，弹性工作,职位描述：<p>工作职责:<p>1. 参与中文分词、新词发现、词性标注、文本分类聚类、知识库...,北京 - 朝阳区 - 望京 - 来广营朝来科技产业园18号院18号楼,凤凰金融,金融,A轮,150-500人,NaN,https://www.lagou.com/gongsi/41483.html,http://www.fengjr.com/,https://www.lagou.com/jobs/2853421.html
3,图像数据挖掘工程师（美颜...,16k-32k /北京 / 经验不限 / 本科及以上 / 全职,算法<p>数据挖掘<p>机器学习<p>深度学习<p>视频,海量数据，技术氛围好，上升空间大,职位描述：<p>部门介绍：<p>1. 高速发展的美图公司（产品包括美图秀秀，美拍，...,北京 - 海淀区 - 五道口 - 优盛大厦D座19层,美图公司,移动互联网，硬件,上市公司,500-2000人,NaN,https://www.lagou.com/gongsi/23291.html,http://hr.meitu.com/,https://www.lagou.com/jobs/4074586.html
4,数据/数据挖掘工程师,20k-40k /北京 / 经验3-5年 / 本科及以上 / 全职,NLP<p>hadoop<p>数据挖掘<p>机器学习<p>Java,大牛云集，明星团队,职位描述：<p>岗位职责：<p>1、负责小爱数据平台的建设，接入和管理小爱各业务数据，进行数...,北京 - 海淀区 - - 清河,小米,移动互联网,D轮及以上,2000人以上,NaN,https://www.lagou.com/gongsi/520.html,http://www.mi.com/,https://www.lagou.com/jobs/4171197.html
5,推荐算法/机器学习/数据挖...,25k-50k /北京 / 经验3-5年 / 本科及以上 / 全职,数据<p>算法<p>数据挖掘,团队分享、部门旅游、晋升空间、奖金丰厚,职位描述：<p>岗位职责：<p>1. 负责网易门户推荐系统、智能匹配等业务的建设<p>2. ...,- 西北旺东路10号院 中关村软件园二期西区7号 网易(北京)公司,网易,电子商务，移动互联网,上市公司,2000人以上,NaN,https://www.lagou.com/gongsi/329.html,http://hr.163.com/,https://www.lagou.com/jobs/2113031.html
6,搜狗输入法-资深数据挖掘...,35k-55k /北京 / 经验5-10年 / 硕士及以上 / 全职,社交<p>专家<p>数据挖掘<p>大数据<p>数据分析,亿万数据挖掘，助理搜狗实现大数据战略,职位描述：<p>项目介绍<p>国内第二大互联网产品，输入法行业的绝对领先者。我们一方面不断优...,北京 - 海淀区 - 五道口 - 中关村东路一号院清华科技园搜狐网络大厦,搜狗,移动互联网，数据服务,C轮,2000人以上,NaN,https://www.lagou.com/gongsi/1537.html,http://www.sogou.com/,https://www.lagou.com/jobs/4019667.html
7,餐饮生态数据挖掘资深工程师,18k-35k /北京 / 经验3-5年 / 本科及以上 / 全职,资深<p>高级<p>中级<p>数据挖掘<p>大数据<p>机器学习,重要业务，成长空间大,职位描述：<p>【工作职责】<p>1、对接餐饮生态收银业务需求，挖掘数据价值，驱动业务成长<...,北京 - 朝阳区 - 望京 - 北京市朝阳区望京东路4号恒基伟业C座（美团点评北京总部）,美团点评,移动互联网，O2O,D轮及以上,2000人以上,NaN,https://www.lagou.com/gongsi/50702.html,http://www.meituan.com/,https://www.lagou.com/jobs/3747809.html
8,风控反欺诈数据挖掘&...,15K-30K /北京 / 经验3-5年 / 本科及以上 / 全职,C/C++<p>数据挖掘<p>机器学习,BAT,职位描述：<p>岗位职责：<p>-针对海量用户行为和内容信息建模，构建和优化用户金融属性画像...,北京 - 海淀区 - 西北旺 - 百度科技园,百度,移动互联网，数据服务,不需要融资,2000人以上,NaN,https://www.lagou.com/gongsi/1575.html,http://talent.baidu.com/external/baidu/index.html,https://www.lagou.com/jobs/3946235.html
9,地图内容生态部_数据挖掘...,15k-25k /北京 / 经验1-3年 / 本科及以上 / 全职,图像处理<p>数据挖掘<p>机器学习<p>深度学习<p>大数据<p>地图,上市公司，绩效加薪，发展空间大，免费班车,职位描述：<p>工作职责<p>-POI数据处理策略的设计与研发<p>-根据实际需求和问题，应...,北京 - 海淀区 - 百度奎科科技大厦,百度,移动互联网，数据服务,不需要融资,2000人以上,NaN,https://www.lagou.com/gongsi/1575.html,http://talent.baidu.com/external/baidu/index.html,https://www.lagou.com/jobs/3803907.html
